# Modèle GINN

Dans ce notebook, on s'intéresse à l'amélioration éventuelle des performances d'un LSTM classique à l'aide du modèle GINN présenté dans le papier: https://arxiv.org/pdf/2410.00288 \
Nous utilisons ici également les rendements logarithmiques. \
L'architecture GINN que nous avons choisie est similaire à celle du papier:
un modèle GARCH (1,1) avec moyenne constante qui nous fournit des prédictions de volatilité et de moyenne \
un calcul de la "vraie" volatilité à partir de la moyenne calculée par le modèle GARCH \
un modèle LSTM qui effectue des prédictions de volatilité à partir des "vraies" volatilités et de la volatilité prédite par le GARCH en utilisant une fonction de coût customisée


In [ ]:
!pip install arch
from arch import arch_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 33.4 MB/s eta 0:00:00


In [ ]:
!pip install yfinance

import tensorflow as tf
import os
from tensorflow.keras import layers, models,Input,regularizers
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
import numpy as np

## Chargement des données et division train/test

In [ ]:
def load_data(symbol="SPY", start="1994-01-01", end="2021-01-01", interval='1d'): #1 donnée=1jour
    df = yf.download(symbol, start=start, end=end, interval=interval)
    df.reset_index(inplace=True)
    return df

def preprocess_data(df):
    df = df[['Date', 'Close']]
    df.dropna(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def train_test(list_df, train_size=0.7, test_size=0.3):
    result = []
    for df in list_df:
        X = df.index.values.reshape(-1, 1)
        y = df['Close'].values

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=train_size, test_size=test_size, shuffle=False
        )
        result.append([X_train, X_test, y_train, y_test])
    return result

def etl_pipeline(train_size=0.7, test_size=0.3):
    # Chargement
    gspc = load_data('^GSPC')
    dji = load_data('^DJI')
    nyse = load_data('^NYA')

    # Preprocess
    gspc_close = preprocess_data(gspc)
    dji_close = preprocess_data(dji)
    nyse_close = preprocess_data(nyse)

    list_df_close = [gspc_close, dji_close, nyse_close]

    # Pour faire le split et shift la data de 90 jours
    def train_test_shift(df, train_size, shift_days=90):
        # Split
        train_size = int(len(df) * train_size)
        X_train = df.iloc[:train_size]
        X_test = df.iloc[train_size:]

        # Shift
        y_train = df.iloc[shift_days:train_size + shift_days]
        y_test = df.iloc[train_size + shift_days:]

        X_train = X_train.iloc[:-shift_days]
        X_test = X_test.iloc[:-shift_days]

        return X_train, X_test, y_train, y_test

    # Appliquer train_test_shift à tous les dataframes
    splits = [train_test_shift(df, train_size) for df in list_df_close]

    return {
        "GSPC": {"X_train": splits[0][0], "X_test": splits[0][1], "y_train": splits[0][2], "y_test": splits[0][3]},
        "DJI":  {"X_train": splits[1][0], "X_test": splits[1][1], "y_train": splits[1][2], "y_test": splits[1][3]},
        "NYSE": {"X_train": splits[2][0], "X_test": splits[2][1], "y_train": splits[2][2], "y_test": splits[2][3]},
    }


Nous avons choisi 90 jours de shift comme dans l'article

In [ ]:
gspc=load_data('^GSPC')
dji=load_data('^DJI')
nyse=load_data('^NYA')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
gspc_close=preprocess_data(gspc)
dji_close=preprocess_data(dji)
nyse_close=preprocess_data(nyse)

<ipython-input-3-47841b545f8f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-3-47841b545f8f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
<ipython-input-3-47841b545f8f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-3-47841b545f8f>:9: SettingWithCopyWarni

In [ ]:
print(gspc_close)

Price             Close
Ticker            ^GSPC
Date                   
1994-01-03   465.440002
1994-01-04   466.890015
1994-01-05   467.549988
1994-01-06   467.119995
1994-01-07   469.899994
...                 ...
2020-12-24  3703.060059
2020-12-28  3735.360107
2020-12-29  3727.040039
2020-12-30  3732.040039
2020-12-31  3756.070068

[6799 rows x 1 columns]


In [ ]:

result = etl_pipeline()

# pour GSPC
X_gspc_train = result["GSPC"]["X_train"]
X_gspc_test  = result["GSPC"]["X_test"]
y_gspc_train = result["GSPC"]["y_train"]
y_gspc_test  = result["GSPC"]["y_test"]

# pour DJI
X_dji_train = result["DJI"]["X_train"]
X_dji_test  = result["DJI"]["X_test"]
y_dji_train = result["DJI"]["y_train"]
y_dji_test  = result["DJI"]["y_test"]

# pour NYSE
X_nyse_train = result["NYSE"]["X_train"]
X_nyse_test  = result["NYSE"]["X_test"]
y_nyse_train = result["NYSE"]["y_train"]
y_nyse_test  = result["NYSE"]["y_test"]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-3-47841b545f8f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-3-47841b545f8f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
<ipython-input-3-47841b545f8f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

In [ ]:
print(X_dji_train)

Price              Close
Ticker              ^DJI
Date                    
1994-01-03   3756.600098
1994-01-04   3783.899902
1994-01-05   3798.820068
1994-01-06   3803.879883
1994-01-07   3820.770020
...                  ...
2012-07-10  12653.120117
2012-07-11  12604.530273
2012-07-12  12573.269531
2012-07-13  12777.089844
2012-07-16  12727.209961

[4669 rows x 1 columns]


In [ ]:
print(y_dji_train)

Price              Close
Ticker              ^DJI
Date                    
1994-05-12   3652.840088
1994-05-13   3659.679932
1994-05-16   3671.500000
1994-05-17   3720.610107
1994-05-18   3732.889893
...                  ...
2013-04-01  14572.849609
2013-04-02  14662.009766
2013-04-03  14550.349609
2013-04-04  14606.110352
2013-04-05  14565.250000

[4759 rows x 1 columns]


# Modèle GARCH
Nous implémentons ici le modèle garch (1,1) à moyenne constante après avoir calculé les log returns de notre série temporelle


In [ ]:
from statsmodels.tsa.stattools import adfuller
from arch import arch_model
import numpy as np
import pandas as pd

# log returns
def calculate_log_returns(prices):
    return np.log(prices / prices.shift(1)).fillna(0)

# GARCH predictions
def garch_predictions(prices_series, forecast_horizon=1):
    mean_predictions = []
    volatility_predictions = []
    prediction_dates = []
    true_volatility = []

    prices_series = prices_series.sort_index()

    # Calcul des log returns
    log_returns = calculate_log_returns(prices_series)

    # Au cas où il y ait un pb de données
    if np.all(log_returns == log_returns.iloc[0]):
        raise ValueError("Log returns are constant. Check the input data.")

    # ADF test pour checker la stationnarité de la série (hypothèse dans le cadre du modèle garch)
    adf_result = adfuller(log_returns.dropna())
    print(f"ADF Statistic: {adf_result[0]}")
    print(f"p-value: {adf_result[1]}")

    if adf_result[1] > 0.05:
        print("Warning: Log returns may not be stationary.")
    else:
        print("Log returns are not stationnary at the confidence level 0.05")

    for i in range(90, len(prices_series)):
        window_prices = prices_series.iloc[i-90:i]
        window_log_returns = calculate_log_returns(window_prices)

        # Fitting
        model = arch_model(window_log_returns, mean='constant', vol='GARCH', p=1, q=1, rescale=True) #Le rescaling des données est fortement recommandé pour améliorer les perfs du modèle
        options = {
            'maxiter': 1000,  # Nombre max d'itérations
            'ftol': 1e-6,     # Tolérance  pour convergence
        }
        model_fit = model.fit(disp='off',options=options)

        # Forecast mean et variance
        forecast = model_fit.forecast(horizon=forecast_horizon)
        predicted_mean = forecast.mean.iloc[-1].values[0]
        predicted_volatility = forecast.variance.iloc[-1].values[0]

        mean_predictions.append(predicted_mean)
        volatility_predictions.append(predicted_volatility)
        prediction_dates.append(prices_series.index[i])

        #Calcul de "true" volatility
        current_log_return = np.log(prices_series.iloc[i] / prices_series.iloc[i-1])
        true_volatility.append((current_log_return - predicted_mean)**2 )

    # Sortie:
    predictions_df = pd.DataFrame({
        'Date': prediction_dates,
        'Predicted_Mean': mean_predictions,
        'Predicted_Volatility': volatility_predictions,
        'True_Volatility': true_volatility
    }).set_index('Date')

    return predictions_df

#Architecture du LSTM
L'architecture implémentée est celle décrite dans l'article

In [ ]:
#Modèle LSTM
def create_lstm_model():
    model = models.Sequential([
        Input(shape=(90, 1)),  # Define input shape here
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256),
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dense(1, kernel_regularizer=regularizers.l2(1e-4))
    ])
    return model





Exécution du modèle garch sur notre train test

In [ ]:

# Calcul parallèle pour accélérer
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["TF_NUM_INTRAOP_THREADS"] = "8"
os.environ["TF_NUM_INTEROP_THREADS"] = "8"

tf.config.threading.set_intra_op_parallelism_threads(8)
tf.config.threading.set_inter_op_parallelism_threads(8)

prices_series = X_dji_train.iloc[:,0]
predictions_df = garch_predictions(prices_series)

true_volatility = predictions_df['True_Volatility'].values
predicted_volatility = predictions_df['Predicted_Volatility'].values

print(true_volatility)
print(predicted_volatility)


ADF Statistic: -15.829291476493498
p-value: 9.987269738330167e-29
[1.14372335e-04 8.02101321e-05 2.12485295e-04 ... 6.80310767e-04
 3.71922090e-04 2.16250707e-05]
[0.56489893 0.54862113 0.49840484 ... 0.89146804 0.94036626 0.98801149]


In [ ]:
print(predictions_df)

            Predicted_Mean  Predicted_Volatility  True_Volatility
Date                                                             
1994-05-12       -0.004158              0.564899         0.000114
1994-05-13       -0.007085              0.548621         0.000080
1994-05-16       -0.011352              0.498405         0.000212
1994-05-17       -0.009317              0.474618         0.000511
1994-05-18       -0.002033              0.629019         0.000028
...                    ...                   ...              ...
2012-07-10       -0.020004              0.945773         0.000181
2012-07-11       -0.025942              0.922671         0.000488
2012-07-12       -0.028566              0.891468         0.000680
2012-07-13       -0.003205              0.940366         0.000372
2012-07-16        0.000739              0.988011         0.000022

[4579 rows x 3 columns]


On prépare les données pour les fournir au lstm (reshaping+scaling+conversion en flottants et en tableaux numpy)

In [ ]:
from sklearn.preprocessing import StandardScaler

#"Vraie" volatilité issue de l'estimation de la moyenne par le modèle garch : (r_t-mu_t)**2
true_volatility = predictions_df['True_Volatility'].values

# volatilité prédite issue du modèle garch
predicted_volatility = predictions_df['Predicted_Volatility'].values

# Preparer la data pour le LSTM
X = np.array([true_volatility[i-90:i] for i in range(90, len(true_volatility))], dtype=np.float32).reshape(-1, 90, 1)
y = np.vstack((true_volatility[90:], predicted_volatility[90:])).astype(np.float32).T  # Shape: [n_samples, 2]

# Standardize X pour de meilleures perfs
scaler_X = StandardScaler()
X_scaled = np.array([scaler_X.fit_transform(seq.reshape(-1, 1)) for seq in X], dtype=np.float32)  # Reshape each sequence to 2D before scaling

# Standardize y pour de meilleures perfs
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)  # Standardize y

# Check
print(f"X shape: {X_scaled.shape}, X dtype: {X_scaled.dtype}")
print(f"y shape: {y_scaled.shape}, y dtype: {y_scaled.dtype}")




X shape: (4489, 90, 1), X dtype: float32
y shape: (4489, 2), y dtype: float32


On définit la fonction cout customisée que l'on va utiliser ici:

In [ ]:


# Fonction coût utilisée lors de l'entraînement:

def custom_loss(predicted_volatility, lambda_param=0.5):
    predicted_volatility = tf.convert_to_tensor(predicted_volatility, dtype=tf.float32)

    def loss(y_true, y_pred):
        batch_size = tf.shape(y_true)[0]
        predicted_vol_batch = predicted_volatility[:batch_size]

        mse = tf.keras.losses.MeanSquaredError()
        mse_true = mse(y_true, y_pred)
        mse_garch = mse(predicted_vol_batch, y_pred)

        return lambda_param * mse_true + (1 - lambda_param) * mse_garch

    return loss


# Initialisation et Compilation
model = create_lstm_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)  # Clip gradients with L2 norm threshold = 1.0
model.compile(optimizer=optimizer, loss=custom_loss(y_scaled[:,1],lambda_param=0.01))

# Entrainement
model.fit(X_scaled, y_scaled[:, 0], epochs=10, batch_size=32)

Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - loss: 0.0921
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0221
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0219
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0187
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0207
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0188
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0187
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0189
Epoch 9/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0181
Epoch 10/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0188


On prédit les volatilités par le lstm

In [ ]:

y_pred_scaled = model.predict(X_scaled)  # Shape: (n_samples, 1)
print(y_pred_scaled)


141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
[[-0.4587227 ]
 [-0.45028377]
 [-0.44378483]
 ...
 [-0.3782543 ]
 [-0.37854367]
 [-0.37898362]]


On enlève le scaling sur nos prédictions

In [ ]:
# Inverse transform du scaling vers le scale original


dummy_array = np.zeros_like(y_scaled)
dummy_array[:, 1] = y_pred_scaled.flatten()  # Replace the second column with predicted volatilities

# Inverse transform
y_pred_inverse = scaler_y.inverse_transform(dummy_array)  # Shape: (n_samples, 2)

On affiche les résultats avec les log_returns issus de la prédiction

In [ ]:

predicted_volatility_lstm_original_scale = y_pred_inverse[:, 1]  # Shape: (n_samples,)

# Step 3: Combiner predicted volatilities par le LSTM avec les moyennes correspodnantes
means = predictions_df['Predicted_Mean'].values[90:]  # Shape: (n_samples,)

# Calculer log_returns issus de la prédiction par le LSTM
log_returns_pred = means + np.sqrt(predicted_volatility_lstm_original_scale)

output_df = pd.DataFrame({
    'Date': predictions_df.index[90:],  # Use the corresponding dates
    'Predicted_Mean': means,
    'Predicted_Volatility GINN': predicted_volatility_lstm_original_scale,
    'Log_Return': log_returns_pred
})

# output DataFrame
print(output_df)

           Date  Predicted_Mean  Predicted_Volatility GINN  Log_Return
0    1994-09-20        0.093853                   0.358098   -0.367785
1    1994-09-21        0.077710                   0.376151   -0.369584
2    1994-09-22        0.074503                   0.390054   -0.358040
3    1994-09-23        0.057090                   0.397799   -0.374252
4    1994-09-26        0.049137                   0.395731   -0.388300
...         ...             ...                        ...         ...
4484 2012-07-10       -0.020004                   0.531452   -0.343894
4485 2012-07-11       -0.025942                   0.530790   -0.352950
4486 2012-07-12       -0.028566                   0.530238   -0.357234
4487 2012-07-13       -0.003205                   0.529619   -0.322212
4488 2012-07-16        0.000739                   0.528678   -0.317672

[4489 rows x 4 columns]


In [ ]:
print(y[:,0])

[1.2361291e-02 6.7635365e-03 6.1256648e-03 ... 6.8031077e-04 3.7192210e-04
 2.1625070e-05]


In [ ]:
print(predicted_volatility)

[0.56489893 0.54862113 0.49840484 ... 0.89146804 0.94036626 0.98801149]


In [ ]:
print(calculate_log_returns(dji_close.iloc[180:,:]))

NameError: name 'calculate_log_returns' is not defined